## 思路
场景：现有一子弹由不同仪器测量，得到多个数据表，数据表字段有 T	x	y	z	Vx	Vy	Vz	V	Ax	Ay	Az	A 代表的含义是：时间，子弹在某时刻三维空间不同坐标位置，不同方向上的速度、速度、三维轴上的加速度，子弹加速度。现在需据某算法建立出拟合函数，从三个维度：速度、加速度、位移三个方向输出其与拟合的子弹运行轨迹数据的标准差，并将其新建三列输出至原始数据表里。
算法设计目前考虑：  

1. 加权平均法
* 读取所有数据表，并将其存储为 Pandas DataFrame 的形式。
* 对于每个数据表，计算每个子弹的平均速度和平均加速度。这可以通过计算每个子弹所有时间点上的速度和加速度的平均值来完成。
* 将每个子弹的平均速度和平均加速度作为该仪器的性能评估指标。对于每个仪器，计算所有子弹的平均速度和平均加速度的平均值，作为该仪器的总体性能评估指标。
* 选择具有最高总体性能评估指标的仪器作为最优仪器。


2. 算法  
* 数据预处理：将每个数据表中的数据合并成一个大的数据集，并根据时间戳进行排序。
* 特征提取：使用统计学和机器学习方法提取每个数据点的特征，例如平均速度、加速度、位置等。
* 模型选择：选择一个适当的模型，该模型可以根据**提取的特征来预测子弹运动轨迹**。可以尝试不同的模型，如决策树、随机森林、神经网络等，并根据交叉验证或其他方法来评估模型的性能。
* 参数优化：根据模型选择的结果，对模型进行参数调整和优化，以提高模型的准确性和泛化能力。
* 最优仪器选择：使用训练好的模型来预测每个数据点的子弹运动轨迹，并根据预测结果来确定最优的仪器。
* 遍历每个数据表，根据预测误差来确定最优的仪器。在循环中，首先读取数据表，然后对数据表进行特征提取。接下来，我们将特征作为输入数据，使用训练好的随机森林模型来预测每个数据点的子弹运动轨迹。最后，我们计算预测结果与实际值之间的均方根误差（RMSE），并将其与之前的最小误差进行比较。如果当前误差更小，则将当前仪器设为最优仪器。

In [1]:
import numpy as np
import pandas as pd

data = pd.read_excel("./instrument.xlsx")
columns = data.columns.tolist()
columns

['T', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'V', 'Ax', 'Ay', 'Az', 'A']

In [2]:
def create_new_scheme(nums, file_name):
    """
    创建方案的函数
    :param nums: 创建方案的个数
    :param file_name: 原始方案路径，xlsx格式
    """
    data = pd.read_excel(file_name)
    save_name = file_name.split(".")[0]
    i = 0
    while i < nums:
        n = np.random.randint(10)
        dd = data.copy()
        # if np.random.rand() > 0.5:
        for col in columns:
            if col == 'T':
                continue
            dd[col] = dd[col].map(lambda x:x + n * np.random.rand())
        dd.to_excel(save_name + f"instrument{i}.xlsx", index=0)
        i += 1
        
create_new_scheme(10, "./instrument.xlsx")

#### 算法细节：
数据预处理：对每个数据表进行清洗、去重、填充缺失值等预处理工作，确保数据的质量。  
特征工程：从原始数据中提取相关的特征，例如速度的平均值、加速度的标准差等。  
模型选择：选择合适的机器学习模型，例如随机森林等。  
模型训练：使用训练数据集对模型进行训练，并调整模型参数以获得最佳性能。  
模型评估：使用测试数据集对模型进行评估，计算预测误差等指标，并根据结果调整模型。  
模型应用：使用训练好的模型对新数据进行预测，并根据预测结果进行后续分析和决策。  

将数据集中的时间、位置、速度、加速度和总加速度作为特征，将拟合的位置、速度、加速度作为目标，使用随机森林模型进行训练和预测。在训练过程中，可以使用交叉验证等方法来评估模型性能和调整模型参数，以提高预测准确性。
增加目标变量位移的预测能力，可以在每个仪器的特征工程处理中新增一个特征变量“displacement”，通过计算位置坐标（x、y、z）分别间隔1时刻所产生的位移，并根据勾股定理求出该时刻间任意三维坐标点之间的欧式距离来得到这个值

In [3]:
import os
import glob
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 读取数据表
# data_files = ["instrument1.xlsx", "instrument2.xlsx", "instrument3.xlsx"]
data_files = glob.glob("./*.xlsx")
data_tables = []
for file in data_files:
    data_table = pd.read_excel(file)
    data_tables.append(data_table)
print(data_tables)
    
# df_list = []
# for i in range(1, num_tables + 1):
#     df_list.append(pd.read_csv(f"table_{i}.csv"))

# df = pd.concat(df_list)
# df = df.sort_values(by=["T"])

# 特征工程
for i, table in enumerate(data_tables):
    table["speed"] = (table["Vx"] ** 2 + table["Vy"] ** 2 + table["Vz"] ** 2) ** 0.5
    table["acceleration"] = (table["Ax"] ** 2 + table["Ay"] ** 2 + table["Az"] ** 2) ** 0.5
    # 计算位移
    table["dx"] = table["x"].diff().fillna(0)
    table["dy"] = table["y"].diff().fillna(0)
    table["dz"] = table["z"].diff().fillna(0)
    table["displacement"] = (table["dx"] ** 2 + table["dy"] ** 2 + table["dz"] ** 2) ** 0.5
    print(table)

# 模型训练和评估
min_error = float("inf")
best_instrument = None
std_dev_list = [] # 新增：用于存储每个数据集的标准差
for i, table in enumerate(data_tables):
    # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split(table[["T", "x", "y", "z", "speed", "acceleration", "displacement"]],
                                                        table[["Vx", "Vy", "Vz", "Ax", "Ay", "Az", "dx", "dy", "dz"]],
                                                        test_size=0.3,
                                                        random_state=42)
    # 添加位移的预测能力

    # 训练模型
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # 测试模型
    y_pred = rf.predict(X_test)
    error = mean_squared_error(y_test, y_pred, squared=False)
    print('--------------------------------------------------------------------------------------')
    print(f"RMSE: {error}") 
#     print(y_pred)
#     print(y_pred[:, :3])
#     print('--------------------------------------------------------------------------------------')
#     print(y_test)
    # 计算速度误差
    speed_error = mean_squared_error(y_test[["Vx", "Vy", "Vz"]], y_pred[:, :3], squared=False)
    print(f"Speed RMSE: {speed_error}")
    
    # 计算加速度误差
    acceleration_error = mean_squared_error(y_test[["Ax", "Ay", "Az"]], y_pred[:, 3:6], squared=False)
    print(f"Acceleration RMSE: {acceleration_error}")

    # 计算位移误差
    displacement_error = mean_squared_error(y_test[["dx", "dy", "dz"]], y_pred[:, -3:], squared=False)
    print(f"Displacement RMSE: {displacement_error}")

    
    # 新增：计算总方差，然后开平方得到标准差，并将其添加到 std_dev_list 中
    total_variance = (speed_error ** 2 + acceleration_error ** 2 + displacement_error ** 2) / 3
    std_dev = total_variance ** 0.5
    std_dev_list.append(std_dev)
    
    
    
    
    # 更新最小误差和最优仪器
    if error < min_error:
        min_error = error
        best_instrument = i

# print(f"Best instrument: {best_instrument}")
print(f"Best instrument: {data_files[best_instrument]}")
print("Standard deviations for each dataset:")
for i, std_dev in enumerate(std_dev_list):
    print(f"{data_files[i]}: {std_dev}")

[       T              x              y             z         Vx         Vy  \
0      1  440773.328417 -103444.679666  1.147018e+06  18.345410 -68.211285   
1      2  440788.331240 -103512.595016  1.146939e+06  18.269808 -69.062432   
2      3  440805.478820 -103592.727904  1.146855e+06  18.203382 -76.243122   
3      4  440819.964181 -103668.691571  1.146777e+06  22.864841 -74.740966   
4      5  440833.442605 -103747.754725  1.146693e+06  22.665973 -74.268306   
..   ...            ...            ...           ...        ...        ...   
96    97  442390.242928 -111760.291924  1.140935e+06  22.281262 -92.974832   
97    98  442409.306308 -111854.703196  1.140889e+06  23.292430 -95.407811   
98    99  442428.179546 -111952.411978  1.140848e+06  23.716348 -94.709442   
99   100  442445.642348 -112045.450577  1.140801e+06  22.462568 -91.298229   
100  101  442462.344468 -112145.324530  1.140757e+06  23.562426 -94.437772   

            Vz           V        Ax        Ay        Az      

--------------------------------------------------------------------------------------
RMSE: 3.600018081588362
Speed RMSE: 0.45988837488356205
Acceleration RMSE: 0.09585001917435093
Displacement RMSE: 10.244315850707173
--------------------------------------------------------------------------------------
RMSE: 5.2335894200437965
Speed RMSE: 2.3866638565008302
Acceleration RMSE: 2.4477456232360177
Displacement RMSE: 10.866358780394544
--------------------------------------------------------------------------------------
RMSE: 4.998699832738855
Speed RMSE: 2.1874520946678295
Acceleration RMSE: 2.136674621186934
Displacement RMSE: 10.671972782361804
--------------------------------------------------------------------------------------
RMSE: 3.600018081588362
Speed RMSE: 0.45988837488356205
Acceleration RMSE: 0.09585001917435093
Displacement RMSE: 10.244315850707173
--------------------------------------------------------------------------------------
RMSE: 5.427047349726918
Speed RMSE: 2

通过计算所有时间步长的距离（V、A、拟合曲线上最近点之间）与其平均值的偏差来获得标准差。

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 读取数据表
data_files = glob.glob("./*.xlsx")
data_tables = []
for file in data_files:
    data_table = pd.read_excel(file)
    data_tables.append(data_table)

# 特征工程
for i, table in enumerate(data_tables):
    table["speed"] = (table["Vx"] ** 2 + table["Vy"] ** 2 + table["Vz"] ** 2) ** 0.5
    table["acceleration"] = (table["Ax"] ** 2 + table["Ay"] ** 2 + table["Az"] ** 2) ** 0.5

# 模型训练和评估
min_error = float("inf")
best_instrument = None
metrics = []

for i, table in enumerate(data_tables):
    # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split(table[["T", "x", "y", "z", "speed", "acceleration"]],
                                                        table[["Vx", "Vy", "Vz", "Ax", "Ay", "Az"]],
                                                        test_size=0.3,
                                                        random_state=42)

    # 训练模型
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # 测试模型
    y_pred = rf.predict(X_test)
    error = mean_squared_error(y_test, y_pred, squared=False)

    # 计算速度、加速度和轨迹的标准差
    speed_std = np.std(y_pred[:, :3], axis=0)
    acceleration_std = np.std(y_pred[:, 3:], axis=0)
    trajectory_std = np.std(y_pred[:, :2], axis=0)

    # 计算综合评分
    combined_score = np.mean([speed_std, acceleration_std, trajectory_std])

    # 更新最小误差和最优仪器
    if combined_score < min_error:
        min_error = combined_score
        best_instrument = i

    # 记录评估指标
    metrics.append((speed_std, acceleration_std, trajectory_std))

print(f"Best instrument: {data_files[best_instrument]}")
print(f"Speed std: {metrics[best_instrument][0]}")
print(f"Acceleration std: {metrics[best_instrument][1]}")
print(f"Trajectory std: {metrics[best_instrument][2]}")

对于每个数据表，我们将输入特征 X 和输出特征 y（包括速度和加速度）分别传入模型进行训练，得到两个不同的高斯过程回归模型：一个用于预测速度，一个用于预测加速度。然后，我们用测试数据进行预测，并计算每个数据点与拟合曲线之间的标准差。最后，我们将预测结果保存到新的数据表中。

这段代码的逻辑如下：

1. 导入所需库：os, glob, pandas, 和 scikit-learn 的 RandomForestRegressor, mean_squared_error, train_test_split。

2. 读取数据表：从当前目录下读取所有 .xlsx 文件，并将每个文件的数据表读入一个 Pandas DataFrame，存储在 data_tables 列表中。

3. 特征工程：对于每个数据表，计算速度、加速度和位移等特征，将这些特征添加到原始数据表中。

4. 模型训练和评估：遍历每个数据表，执行以下操作：  
   a. 划分数据集：将数据表拆分为训练集和测试集，其中训练集包含70%的数据，测试集包含30%的数据。输入特征为时间、位置、速度、加速度和位移，输出特征为速度、加速度和位移的分量。  
   b. 训练模型：使用训练集训练一个随机森林回归模型。  
   c. 测试模型：使用测试集评估模型，计算预测值与实际值之间的均方根误差 (RMSE)。  
   d. 计算速度、加速度和位移的 RMSE。  
   e. 更新最小误差和最优仪器：如果当前数据表的误差小于之前的最小误差，则更新最小误差和最优仪器。
  
5. 输出最优仪器：打印最优仪器的文件名。

随机森林拟合曲线的过程可以类比为众多决策树“互相协作”来完成这项任务。每个决策树都是一种基本的预测模型，它们独立地对数据进行学习和预测，并输出一个单独的结果。随机森林会将所有基本模型的结果进行汇总、整合，最终生成一个同时考虑了所有决策树结果并得出符合实际情况较好且平滑（光滑）形态结构的连续性函数式，即所谓拟合曲线。

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# 数据预处理
# 对每个数据表进行清洗、去重、填充缺失值等预处理工作，确保数据的质量
data = []
data_files = glob.glob("./*.xlsx")
# data_tables = []
# for file in data_files:
#     data_table = pd.read_excel(file)
#     data_tables.append(data_table)
# 数据读取与特征工程
for file in data_files:
    # 读取数据表
    df = pd.read_excel(file)
    # 特征提取
#     X = df[["T", "x", "y", "z"，"Dx", "Dy", "Dz"]].values
#     y = df[["Vx", "Vy", "Vz", "Ax", "Ay", "Az", "A", "Dx", "Dy", "Dz"]].values
    X = df[["T", "x", "y", "z"]].values
    y = df[["Vx", "Vy", "Vz", "Ax", "Ay", "Az", "A"]].values
    # 将速度、加速度和位移分开，用于模型训练和预测
    y_vel = y[:, :3]
    y_acc = y[:, 3:6]
#     y_disp = y[:, 6:9]
    # 将时间作为索引
    df.set_index("T", inplace=True)
    # 将每个数据表加入数据列表中
    data.append((X, y_vel, y_acc,  df))

num_tables = len(data_files)
# 数据划分
train_data = data[:int(num_tables*0.7)]
test_data = data[int(num_tables*0.3):]

# 定义高斯过程回归模型
# kernel 采用了 RBF 核和 WhiteKernel 噪声核，可通过调整 length_scale 和 noise_level 参数控制模型的灵活度和泛化能力。
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0)  
gpr = GaussianProcessRegressor(kernel=kernel)

# 对每个数据表进行模型训练和预测。
# 首先，分别训练速度和加速度模型，然后使用训练好的模型对测试集进行预测，得到预测值和标准差。
# 将预测结果加入测试数据表中，并计算每个数据点与拟合曲线之间的标准差，将预测结果保存到新的数据表中。
for i, (X_train, y_train_vel, y_train_acc, _) in enumerate(train_data):
    # 训练速度和加速度模型
    gpr_vel = gpr.fit(X_train, y_train_vel)
    gpr_acc = gpr.fit(X_train, y_train_acc)

    # 预测速度和加速度
    X_test, y_test_vel, y_test_acc, df = test_data[i]
    y_pred_vel, std_vel = gpr_vel.predict(X_test, return_std=True)
    y_pred_acc, std_acc = gpr_acc.predict(X_test, return_std=True)

#     # 对速度进行积分，得到位移增量
#     dt = np.diff(X_test[:, 0])[:, np.newaxis]
#     delta_pos = y_pred_vel[:-1, :] * dt + 0.5 * y_pred_acc[:-1, :] * dt ** 2

#     # 计算当前的位移
#     pos_pred = np.zeros_like(y_pred_vel)
#     pos_pred[0, :] = train_data.iloc[-1, [1, 2, 3]].values + delta_pos[0, :]
#     for j in range(1, len(delta_pos)):
#         pos_pred[j, :] = pos_pred[j-1, :] + delta_pos[j, :]
    
#     # 计算标准差
#     x0 = X_test[0, 1:] # 初始位置
#     y_test_displacement = X_test[:, 1:] - x0 # 真实位移
#     y_pred_displacement = pos_pred - x0 # 预测位移
#     std_displacement = np.std(y_test_displacement - y_pred_displacement)
    
    
    
    # 计算每个数据点与拟合曲线之间的标准差
    df["std_vel"] = std_vel
    df["std_acc"] = std_acc
#     df_test["std_displacement"] = std_displacement
    

    # 将预测结果加入测试数据表中
    df["Vx_pred"] = y_pred_vel[:, 0]
    df["Vy_pred"] = y_pred_vel[:, 1]
    df["Vz_pred"] = y_pred_vel[:, 2]
    df["Ax_pred"] = y_pred_acc[:, 0]
    df["Ay_pred"] = y_pred_acc[:, 1]
    df["Az_pred"] = y_pred_acc[:, 2]
#     df_test["x_pred"] = pos_pred[:, 0]
#     df_test["y_pred"] = pos_pred[:, 1]
#     df_test["z_pred"] = pos_pred[:, 2]

    # 将预测结果保存到新的数据表中
    df.to_csv(f"result_{i}.csv", index=False)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 读取多张Excel表格
dfs = []
# for i in range(10):  # 假设有10个数据表
#     df = pd.read_excel(f'ins{i}.xlsx')
#     dfs.append(df)
# 读取数据表
# data_files = glob.glob("./*.xlsx")
# for file in data_files:
#     data_table = pd.read_excel(file)
#     dfs.append(data_table)

for i in range(0, 10):
    dfs.append(pd.read_excel(f"instrument{i}.xlsx"))
# 合并多个数据表
data = pd.concat(dfs)
data = data.sort_values(by=["T"])
# 分离特征和目标变量
X = data[['T', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'Ax', 'Ay', 'Az', 'A']]
y_pos = data[['x', 'y', 'z']]
y_vel = data[['Vx', 'Vy', 'Vz']]
y_acc = data[['Ax', 'Ay', 'Az']]


min_error = float("inf")
best_instrument = None
for i, table in enumerate(dfs):
    # 划分训练集和测试集
    X_train, X_test, y_pos_train, y_pos_test, y_vel_train, y_vel_test, y_acc_train, y_acc_test = train_test_split(X, y_pos, y_vel, y_acc, test_size=0.2, random_state=42)

    # 创建随机森林回归模型并训练
    rf_pos = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_pos.fit(X_train, y_pos_train)

    rf_vel = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_vel.fit(X_train, y_vel_train)

    rf_acc = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_acc.fit(X_train, y_acc_train)

    # 预测测试集结果
    y_pos_pred = rf_pos.predict(X_test)
    y_vel_pred = rf_vel.predict(X_test)
    y_acc_pred = rf_acc.predict(X_test)

    # 计算均方误差
    mse_pos = mean_squared_error(y_pos_test, y_pos_pred)
    mse_vel = mean_squared_error(y_vel_test, y_vel_pred)
    mse_acc = mean_squared_error(y_acc_test, y_acc_pred)

    # 打印结果
    print('MSE of position: ', mse_pos)
    print('MSE of velocity: ', mse_vel)
    print('MSE of acceleration: ', mse_acc)

In [ ]:
table["dx"] = table["x"].diff().fillna(0)
table["dy"] = table["y"].diff().fillna(0)
table["dz"] = table["z"].diff().fillna(0)
y_pred = table[["Vx", "Vy", "Vz", "Ax", "Ay", "Az", "dx", "dy", "dz"]]
y_pred

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# 数据预处理
# 对每个数据表进行清洗、去重、填充缺失值等预处理工作，确保数据的质量
data = []
data_files = glob.glob("./*.xlsx")
# data_tables = []
# for file in data_files:
#     data_table = pd.read_excel(file)
#     data_tables.append(data_table)
# 数据读取与特征工程
for file in data_files:
    # 读取数据表
    df = pd.read_excel(file)
    # 特征提取
#     X = df[["T", "x", "y", "z"，"Dx", "Dy", "Dz"]].values
#     y = df[["Vx", "Vy", "Vz", "Ax", "Ay", "Az", "A", "Dx", "Dy", "Dz"]].values
    X = df[["T", "x", "y", "z"]].values
    y = df[["Vx", "Vy", "Vz", "Ax", "Ay", "Az", "A"]].values
    # 将速度、加速度和位移分开，用于模型训练和预测
    y_vel = y[:, :3]
    y_acc = y[:, 3:]
#     y_disp = y[:, 6:9]
    # 将时间作为索引
    df.set_index("T", inplace=True)
    # 将每个数据表加入数据列表中
    data.append((X, y_vel, y_acc,  df))

num_tables = len(data_tables)
# 数据划分
train_data = data[:int(num_tables*0.7)]
test_data = data[int(num_tables*0.3):]

# 定义高斯过程回归模型
# kernel 采用了 RBF 核和 WhiteKernel 噪声核，可通过调整 length_scale 和 noise_level 参数控制模型的灵活度和泛化能力。
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0)  
gpr = GaussianProcessRegressor(kernel=kernel)

# 对每个数据表进行模型训练和预测。
# 首先，分别训练速度和加速度模型，然后使用训练好的模型对测试集进行预测，得到预测值和标准差。
# 将预测结果加入测试数据表中，并计算每个数据点与拟合曲线之间的标准差，将预测结果保存到新的数据表中。
for i, (X_train, y_train_vel, y_train_acc, _) in enumerate(train_data):
    # 训练速度和加速度模型
    gpr_vel = gpr.fit(X_train, y_train_vel)
    gpr_acc = gpr.fit(X_train, y_train_acc)

    # 预测速度和加速度
    X_test, y_test_vel, y_test_acc, df = test_data[i]
    y_pred_vel, std_vel = gpr_vel.predict(X_test, return_std=True)
    y_pred_acc, std_acc = gpr_acc.predict(X_test, return_std=True)

#     # 对速度进行积分，得到位移增量
#     dt = np.diff(X_test[:, 0])[:, np.newaxis]
#     delta_pos = y_pred_vel[:-1, :] * dt + 0.5 * y_pred_acc[:-1, :] * dt ** 2

#     # 计算当前的位移
#     pos_pred = np.zeros_like(y_pred_vel)
#     pos_pred[0, :] = train_data.iloc[-1, [1, 2, 3]].values + delta_pos[0, :]
#     for j in range(1, len(delta_pos)):
#         pos_pred[j, :] = pos_pred[j-1, :] + delta_pos[j, :]
    
#     # 计算标准差
#     x0 = X_test[0, 1:] # 初始位置
#     y_test_displacement = X_test[:, 1:] - x0 # 真实位移
#     y_pred_displacement = pos_pred - x0 # 预测位移
#     std_displacement = np.std(y_test_displacement - y_pred_displacement)
    
#     # 计算位移
#     # dt = 0.1 # 时间间隔
#     dt = np.diff(X_test[:, 0])[:, np.newaxis]
#     v0 = np.zeros(3) # 初始速度
#     x0 = X_test[0, 1:] # 初始位置
#     x_pred = np.zeros_like(y_pred_vel)
#     x_pred[0] = x0
#     for j in range(1, len(X_test)):
#         x_pred[j] = x_pred[j-1] + y_pred_vel[j-1] * dt + 0.5 * y_pred_acc[j-1] * dt**2

#     # 计算标准差
#     y_test_displacement = X_test[:, 1:] - x0 # 真实位移
#     y_pred_displacement = x_pred - x0 # 预测位移
#     std_displacement = np.std(y_test_displacement - y_pred_displacement)
    
    
    
    
    # 计算每个数据点与拟合曲线之间的标准差
    df["std_vel"] = std_vel
    df["std_acc"] = std_acc
#     df_test["std_displacement"] = std_displacement
    

    # 将预测结果加入测试数据表中
    df["Vx_pred"] = y_pred_vel[:, 0]
    df["Vy_pred"] = y_pred_vel[:, 1]
    df["Vz_pred"] = y_pred_vel[:, 2]
    df["Ax_pred"] = y_pred_acc[:, 0]
    df["Ay_pred"] = y_pred_acc[:, 1]
    df["Az_pred"] = y_pred_acc[:, 2]
#     df_test["x_pred"] = pos_pred[:, 0]
#     df_test["y_pred"] = pos_pred[:, 1]
#     df_test["z_pred"] = pos_pred[:, 2]

    # 将预测结果保存到新的数据表中
    df.to_csv(f"test_result_{i}.csv", index=False)

In [ ]:
y_pred[:, 3:6]

In [ ]:
y_pred[:, -3:]

In [ ]:
y_pred

对每个数据集的运动轨迹数据进行拟合，例如使用多项式回归或者样条插值等方法分别对x, y, z轴的位置、速度、加速度进行拟合，得到拟合曲线。然后，对于每个拟合曲线计算实际数据点与拟合曲线之间的差异，即残差。然后计算每个轴上的速度、加速度、位移的残差的标准差。

现需要根据某的算法得到每个数据集从速度、加速度、位移三个方面上与拟合的子弹的运动的误差，怎么设计算法细节和代码实现。

对这个问题，一个比较直观的方法是，首先对每个数据集进行运动轨迹的拟合，例如使用多项式回归或者样条插值等方法，得到拟合曲线。然后，根据这个拟合曲线和原始数据集的数据，计算速度、加速度和位移三个方面上的误差。

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline
from sklearn.metrics import mean_squared_error

# 1. 整理数据
# 假设数据表分别存储为csv文件，这里将它们整合成一个数据集
data_files = ['instrument1.xlsx', 'instrument2.xlsx', 'instrument3.xlsx']
data_list = []

for file in data_files:
    data = pd.read_excel(file)
    data_list.append(data)

all_data = pd.concat(data_list, ignore_index=True)

# 2. 数据预处理
# 假设缺失值以及异常值已处理

# 3. 拟合曲线
def fit_spline(data, x_col, y_col, k=3, s=0.1):
    x = data[x_col]
    y = data[y_col]
    spline = UnivariateSpline(x, y, k=k, s=s)
    return spline

# 对每个子弹的运动轨迹数据进行拟合
splines = {}
for axis in ['x', 'y', 'z']:
    splines[axis] = {
        'position': fit_spline(all_data, 'T', axis),
        'velocity': fit_spline(all_data, 'T', f'V{axis}'),
        'acceleration': fit_spline(all_data, 'T', f'A{axis}')
    }

# 4. 计算标准差
def calculate_std(data, spline, x_col, y_col):
    x = data[x_col]
    y_actual = data[y_col]
    y_predicted = spline(x)
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

std_results = {}
for axis in ['x', 'y', 'z']:
    std_results[axis] = {
        'position': calculate_std(all_data, splines[axis]['position'], 'T', axis),
        'velocity': calculate_std(all_data, splines[axis]['velocity'], 'T', f'V{axis}'),
        'acceleration': calculate_std(all_data, splines[axis]['acceleration'], 'T', f'A{axis}')
    }

# 5. 分析结果
print("标准差结果：")
for axis, results in std_results.items():
    print(f"{axis}-axis:")
    for key, value in results.items():
        print(f"{key}: {value}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit

# 读取数据表
df = pd.read_excel("instrument.xlsx")

# 定义拟合曲线函数
def trajectory_func(t, a, b, c, d, e, f, g, h, i, j):
    x = a + b*t + c*t**2 + d*t**3
    y = e + f*t + g*t**2 + h*t**3
    z = i + j*t
    return x, y, z

# 拟合曲线
popt, pcov = curve_fit(trajectory_func, df['T'], [df['x'], df['y'], df['z'], df['Vx'], df['Vy'], df['Vz'], df['Ax'], df['Ay'], df['Az'], df['A']])

# 计算拟合值
x_fit, y_fit, z_fit = trajectory_func(df['T'], *popt)

# 计算速度、加速度、位移与拟合曲线的标准差
v_std = np.sqrt(mean_squared_error([df['Vx'], df['Vy'], df['Vz']], [popt[1] + 2*popt[2]*df['T'] + 3*popt[3]*df['T']**2,
                                                                popt[4] + 2*popt[5]*df['T'] + 3*popt[6]*df['T']**2,
                                                                popt[8] + popt[9]*df['T']]))
a_std = np.sqrt(mean_squared_error([df['Ax'], df['Ay'], df['Az']], [2*popt[2] + 6*popt[3]*df['T'],
                                                                2*popt[5] + 6*popt[6]*df['T'],
                                                                popt[9]]))
d_std = np.sqrt(mean_squared_error([df['x'], df['y'], df['z']], [x_fit, y_fit, z_fit]))

# 输出结果
print("速度标准差:", v_std)
print("加速度标准差:", a_std)
print("位移标准差:", d_std)

高斯过程回归模型（Gaussian Process Regression，GPR）也可以用于拟合子弹运动轨迹的曲线。GPR是一种基于贝叶斯思想的非参数回归方法，它可以估计任意函数的后验分布，并通过与训练数据的比较来进行预测。相对于多项式回归模型，GPR可以提供更加灵活的模型拟合能力。

具体来说，对于子弹运动轨迹的曲线拟合问题，您可以将每个时间点上子弹的三维坐标作为输入变量，将子弹速度、加速度和位移作为输出变量，然后使用GPR模型来拟合输出变量与输入变量之间的关系。

GPR模型的优点在于可以提供关于预测结果的不确定性估计，这在实际应用中非常有用。此外，GPR模型可以自适应地调整拟合曲线的复杂度，因此它能够有效地处理高维度、非线性和噪声数据。不过，需要注意的是，GPR模型的计算复杂度较高，需要对大规模数据进行适当的优化。

先根据数据表中的位置信息计算出子弹的位移向量，并根据时间信息计算出子弹的平均速度和加速度向量。然后，将这些向量与预测的标准子弹运动的向量进行比较，并计算它们的标准差。